# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [30]:
import numpy as np
import pandas as pd
import math
import xlsxwriter
import requests


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [31]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [32]:
from starter_files.secrets import FMP_FIN_MODEL_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [33]:
symbol = 'AAPL'
api_url = f'https://financialmodelingprep.com/stable/quote?symbol={symbol}&apikey={FMP_FIN_MODEL_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

[{'symbol': 'AAPL', 'name': 'Apple Inc.', 'price': 276.97, 'changePercentage': 0.38055, 'change': 1.05, 'volume': 46901931, 'dayLow': 275.27, 'dayHigh': 280.38, 'yearHigh': 280.38, 'yearLow': 169.21, 'marketCap': 4092606490410, 'priceAvg50': 260.2724, 'priceAvg200': 226.51076, 'exchange': 'NASDAQ', 'open': 275.27, 'previousClose': 275.92, 'timestamp': 1764104400}]


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [34]:
price = data[0]['price']
marketcap = data[0]['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [35]:
my_columns = ['Ticker', 'Stock Price','Market Cap','Number Of Shares to buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Cap,Number Of Shares to buy


In [36]:
final_dataframe._append(
    pd.Series(
    [
        symbol,
        price,
        marketcap,
        'N/A'
    ],
    index=my_columns
    ),
    ignore_index=True
)

/var/folders/97/27my23hx0t5f332mj7xwmnch0000gn/T/ipykernel_85745/3157646817.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe._append(


,Ticker,Stock Price,Market Cap,Number Of Shares to buy
0,AAPL,276.97,4092606490410,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [45]:
my_columns = ['Ticker', 'Stock Price', 'Market Cap', 'Number Of Shares to buy']
all_stock_data = [] # <--- Initialize an empty list to store row dictionaries

for stock in stocks['Ticker'][:10]:
    try:
        api_url = f'https://financialmodelingprep.com/stable/quote?symbol={stock}&apikey={FMP_FIN_MODEL_API_TOKEN}'
        data = requests.get(api_url).json()

        # Check if data is not empty and is a list as expected
        if data and isinstance(data, list):
            # Create a dictionary for the current stock's data
            stock_info = {
                'Ticker': stock,
                'Stock Price': data[0]['price'],
                'Market Cap': data[0]['marketCap'],
                'Number Of Shares to buy': 'N/A'
            }
            # Append the dictionary to the list
            all_stock_data.append(stock_info)
        else:
            print(f"Warning: API returned unexpected data for ticker {stock}")

    except Exception as e:
        print(f"Error processing ticker {stock}: {e}")

# Create the DataFrame ONCE after the loop
final_dataframe = pd.DataFrame(all_stock_data, columns=my_columns)

print(final_dataframe)


Error processing ticker A: Expecting value: line 1 column 1 (char 0)
Error processing ticker AAP: Expecting value: line 1 column 1 (char 0)
Error processing ticker ABC: Expecting value: line 1 column 1 (char 0)
Error processing ticker ABMD: Expecting value: line 1 column 1 (char 0)
Error processing ticker ABT: Expecting value: line 1 column 1 (char 0)
Error processing ticker ACN: Expecting value: line 1 column 1 (char 0)
  Ticker  Stock Price     Market Cap Number Of Shares to buy
0    AAL        13.46     8884730344                     N/A
1   AAPL       276.97  4092606490410                     N/A
2   ABBV       231.80   409640951828                     N/A
3   ADBE       319.55   133764591526                     N/A


In [46]:
print(final_dataframe)

  Ticker  Stock Price     Market Cap Number Of Shares to buy
0    AAL        13.46     8884730344                     N/A
1   AAPL       276.97  4092606490410                     N/A
2   ABBV       231.80   409640951828                     N/A
3   ADBE       319.55   133764591526                     N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: